<a href="https://colab.research.google.com/github/Jamalun/Data_Capstone_Design/blob/master/BERT_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 학습용 데이터셋 불러오기
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab/Kaggle_translated.xlsx', names = ["sentiment","english", "korean" ])
df.head()

,sentiment,english,korean
0,neutral,Technopolis plans to develop in stages an area...,"테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 100,0..."
1,negative,The international electronic industry company ...,국제적인 전자 산업 회사 Elcoteq는 그것의 Talinn 시설로부터 10명의 직...
2,positive,With the new production plant the company woul...,새로운 생산 공장과 함께 회사는 수요의 예상되는 증가를 충족시키기 위하여 그것의 능...
3,positive,According to the company 's updated strategy f...,"2009-2012년 회사의 최신 전략에 따르면, Basware는 순매출의 10%-2..."
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,ASPOCOMP의 파이낸싱은 기술적으로 더 까다로운 HDI 인쇄 회로 기판 PCB에...


# 데이터 라벨링

In [10]:
import numpy as np

In [11]:
labels = []
for sentiment in df['sentiment']:
  if (sentiment == 'neutral'):
    labels.append(33)
  if (sentiment == 'negative'):
    labels.append(0)
  if (sentiment == 'positive'):
    labels.append(1)

df['label'] = labels
  

In [13]:
# 라벨링이 0,1,33로 잘 되었음을 확인
df['label'].unique()

array([33,  0,  1])

In [21]:
# 중립은 제외하고 이진 분류하기
index1 = df[df['label'] == 33].index
df.drop(index1, inplace = True)
df['label'].unique()

array([0, 1])

In [23]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 1573
test shape is: 394


## BERT 시도해보기 !

In [1]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 27.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [24]:
# 문장 추출
sentences = train['korean']
sentences[:10]

931                 " 우리는 이 지역 전체에 대해 상당히 긍정적으로 느끼고 있습니다.
918     Wartsila는 2007년 10월에 계약을 따냈습니다 그리고 그 공장은 러시아의 ...
2313           탈린-헬싱키 노선의 에스토니아인과 리투아니아인 승객 비율도 7월에 증가했다.
1219    핀란드 자동화 솔루션 개발업체인 Cencorp Corporation (OMX Hel...
2192    Tallink는 그들의 전기 시스템을 포함하여, Vana Tallinn과 Regin...
1653    그 서비스는 13 지중해 국가의 사람들이 더 빨리 발트해에 도달하는 것을 허락하기 ...
4095    핀란드의 에너지 회사인 Fortum Oyj는 2007년 11월 13일 그것이 동부 ...
540           영업 이익은 EUR 8.65 mn에서 상승한 EUR 11.07 mn이었습니다.
2281    F-Secure의 통신사업자 ISP, 이동 통신사업자 및 케이블 사업자의 성장은 가...
1706    JP 모건은 스칼라가 2007년 노벨 바이오케어의 성장률 전망치를 현재 34-35p...
Name: korean, dtype: object

In [25]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] " 우리는 이 지역 전체에 대해 상당히 긍정적으로 느끼고 있습니다. [SEP]',
 '[CLS] Wartsila는 2007년 10월에 계약을 따냈습니다 그리고 그 공장은 러시아의 Kostroma 지역에 만투로보에 설치될 것입니다, 러시아, 모스크바의 북동쪽 약 600 km. [SEP]',
 '[CLS] 탈린-헬싱키 노선의 에스토니아인과 리투아니아인 승객 비율도 7월에 증가했다. [SEP]',
 '[CLS] 핀란드 자동화 솔루션 개발업체인 Cencorp Corporation (OMX Helski: CNC1V)은 금요일 (6월 27일) 자사의 운영 재편에 관한 직원 협상을 완료했다고 밝혔습니다. [SEP]',
 '[CLS] Tallink는 그들의 전기 시스템을 포함하여, Vana Tallinn과 Regina Baltica 둘 다의 물샐틈없는 문을 주장합니다. [SEP]',
 '[CLS] 그 서비스는 13 지중해 국가의 사람들이 더 빨리 발트해에 도달하는 것을 허락하기 위하여 의도되었습니다, 단일 운송인과 함께. [SEP]',
 '[CLS] 핀란드의 에너지 회사인 Fortum Oyj는 2007년 11월 13일 그것이 동부 스톡홀름의 Vartan 항구에 바이오 연료로 연소되는 열과 전력 복합 CHP 공장을 건설하는 환경 허가를 받았다고 말했습니다. [SEP]',
 '[CLS] 영업 이익은 EUR 8.65 mn에서 상승한 EUR 11.07 mn이었습니다. [SEP]',
 '[CLS] F-Secure의 통신사업자 ISP, 이동 통신사업자 및 케이블 사업자의 성장은 가장 컸다. [SEP]',
 '[CLS] JP 모건은 스칼라가 2007년 노벨 바이오케어의 성장률 전망치를 현재 34-35pct에서 뿐만 아니라 23-25pct로 낮출 것으로 예상하고 있다. [SEP]']

In [26]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 1, 1, ..., 1, 0, 1])

In [27]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

[CLS] " 우리는 이 지역 전체에 대해 상당히 긍정적으로 느끼고 있습니다. [SEP]
['[CLS]', '"', '우', '##리는', '이', '지역', '전체', '##에', '대해', '상', '##당', '##히', '긍', '##정', '##적으로', '느', '##끼', '##고', '있', '##습', '##니다', '.', '[SEP]']


In [28]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    107,   9604,  26344,   9638,  58939,  96567,  10530,
        33378,   9414,  21928,  18108,   8931,  16605,  17022,   9041,
       118710,  11664,   9647, 119081,  48345,    119,    102,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [29]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [30]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9574,  26784,   9450,  31503,  10892,  20067,   9414,  30134,
         33797,  17751,  11273,    126,    119,    122,    181,  10115,  10459,
          9638, 119188,  11467,  17655,   9952, 107693,  11102,   9761,  17751,
         11273,    121,    119,    124,    181,  10115,  58303,  48345,    119,
           102,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [31]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [32]:
# 문장 추출
sentences = test['korean']
sentences[:10]

1184    Atria는 모스크바 시장에 CampoMos 브랜드를 홍보하기 위하여 9월에 캠페인...
4075    이 부동산 회사는 2007년 같은 분기의 주당 순이익 +α x201a -α 5930...
62      처음에 계약은 2개의 더 많은 유닛을 위한 옵션으로 3척의 페리 보트의 건설을 상상...
2001    중국에서, 현대화와 유지를 위한 엘리베이터, 에스컬레이터, 그리고 해결책을 제공하는...
1193                   일부 주주들은 채권 발행에 찬성표를 던지기로 취소할 수 없다.
4652    핀란드 GeoSentric의 순매출은 2009년 1월부터 3월까지 939,000유로...
874                      세전 이익은 310만 유로에서 2190만 유로로 급증했다.
745                  " 정부는 이 제안을 평가함에 있어 매우 전문적인 접근을 해왔다.
4629    그 회사는 전체 2009년을 위한 그것의 순매출이 2008년 수준 아래로 남아 있기...
1747    세미나에서, 협력에 대한 양해각서가 핀란드 케미라 회사와 베트남 국립 화학 회사 사...
Name: korean, dtype: object

In [33]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] Atria는 모스크바 시장에 CampoMos 브랜드를 홍보하기 위하여 9월에 캠페인을 시작할 것입니다, 그는 말했습니다. [SEP]',
 '[CLS] 이 부동산 회사는 2007년 같은 분기의 주당 순이익 +α x201a -α 5930만 +α x201a -α 0.21과 비교하여 주당 순손실을 기록했다. [SEP]',
 '[CLS] 처음에 계약은 2개의 더 많은 유닛을 위한 옵션으로 3척의 페리 보트의 건설을 상상하였습니다, 총 500 mln 유로 ($ 645.8 mln )의 값에 추가로 확인되었습니다. [SEP]',
 '[CLS] 중국에서, 현대화와 유지를 위한 엘리베이터, 에스컬레이터, 그리고 해결책을 제공하는 핀란드 Kone은 1.5년 안에 베이징과 상하이 사이의 고속 철도 구간에 설치될 342개의 에스컬레이터를 건설할 것입니다. [SEP]',
 '[CLS] 일부 주주들은 채권 발행에 찬성표를 던지기로 취소할 수 없다. [SEP]',
 '[CLS] 핀란드 GeoSentric의 순매출은 2009년 1월부터 3월까지 939,000유로까지 감소했습니다. [SEP]',
 '[CLS] 세전 이익은 310만 유로에서 2190만 유로로 급증했다. [SEP]',
 '[CLS] " 정부는 이 제안을 평가함에 있어 매우 전문적인 접근을 해왔다. [SEP]',
 '[CLS] 그 회사는 전체 2009년을 위한 그것의 순매출이 2008년 수준 아래로 남아 있기를 기대합니다. [SEP]',
 '[CLS] 세미나에서, 협력에 대한 양해각서가 핀란드 케미라 회사와 베트남 국립 화학 회사 사이에 체결되었다. [SEP]']

In [34]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,

In [35]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] Atria는 모스크바 시장에 CampoMos 브랜드를 홍보하기 위하여 9월에 캠페인을 시작할 것입니다, 그는 말했습니다. [SEP]
['[CLS]', 'At', '##ria', '##는', '모', '##스', '##크', '##바', '시', '##장에', 'Campo', '##M', '##os', '브', '##랜드', '##를', '홍', '##보', '##하기', '위하여', '9월', '##에', '캠', '##페인', '##을', '시', '##작', '##할', '것', '##입', '##니다', ',', '그는', '말', '##했', '##습', '##니다', '.', '[SEP]']


In [36]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,  11699,  11021,  11018,   9283,  12605,  20308,  42144,
         9485,  85903,  22188,  11517,  10310,   9376,  42771,  11513,
         9992,  30005,  22440,  68010,  16959,  10530,   9795, 106289,
        10622,   9485,  38709,  14843,   8870,  58303,  48345,    117,
        17889,   9251, 119424, 119081,  48345,    119,    102,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [37]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [38]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,  11699,  11021,  11018,   9283,  12605,  20308,  42144,   9485,
         85903,  22188,  11517,  10310,   9376,  42771,  11513,   9992,  30005,
         22440,  68010,  16959,  10530,   9795, 106289,  10622,   9485,  38709,
         14843,   8870,  58303,  48345,    117,  17889,   9251, 119424, 119081,
         48345,    119,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [39]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [40]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [41]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [42]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [43]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [44]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [45]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [46]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.53
  Training epcoh took: 0:00:56

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.29
  Training epcoh took: 0:00:56

Running Validation...
  Accuracy: 0.88
  Validation took: 0:00:02

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.21
  Training epcoh took: 0:00:56

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:02

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.15
  Training epcoh took: 0:00:56

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:02

Training complete!


In [47]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.89
Test took: 0:00:05
